In [1]:
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from ucimlrepo import fetch_ucirepo
import os
import pdpbox as pdp
from interpret import show
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import numpy as np
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
import statsmodels
import seaborn as sns
import pandas as pd
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import warnings
from lazypredict.Supervised import LazyClassifier
from lazypredict.Supervised import LazyClassifier
from sklearn.datasets import load_breast_cancer
import lazypredict
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.linear_model._logistic import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors._classification import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from pdpbox import pdp, get_dataset, info_plots
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
set_visualize_provider(InlineProvider())
# fetch dataset
car_evaluation = fetch_ucirepo(id=19)
# data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets
feature_names = car_evaluation.feature_names
print(X.columns)

# variable information
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

encoder = ce.OrdinalEncoder(
    cols=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)
# import Random Forest classifier
models_to_include = [
    'svc', 'random_forest', 'kneighbors', 'gradient_boosting', 'decision_tree'
]

data = load_breast_cancer()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.5, random_state=123)
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None,
                     predictions=True,
                     classifiers=[RandomForestClassifier, LogisticRegression, KNeighborsClassifier, MLPClassifier, DecisionTreeClassifier, SVC, xgb.XGBClassifier])
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

Index(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'], dtype='object')


100%|██████████| 7/7 [00:00<00:00,  9.03it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
MLPClassifier,0.99,0.99,0.99,0.99,0.35
LogisticRegression,0.99,0.98,0.98,0.99,0.02
XGBClassifier,0.98,0.98,0.98,0.98,0.10
SVC,0.98,0.98,0.98,0.98,0.02
RandomForestClassifier,0.97,0.97,0.97,0.97,0.22
KNeighborsClassifier,0.96,0.96,0.96,0.96,0.04
DecisionTreeClassifier,0.93,0.93,0.93,0.93,0.01


In [2]:
from tabulate import tabulate

print(type(models))
with open("summary.latext", "w") as f:
    f.write(models.to_latex())

<class 'pandas.core.frame.DataFrame'>


In [3]:
# instantiate the classifier
rfc = RandomForestClassifier(n_estimators=10, random_state=0)


# fit the model
rfc.fit(X_train, y_train)

# Predict the Test set results
y_pred = rfc.predict(X_test)

# Check accuracy score
print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(
    accuracy_score(y_test, y_pred)))
result = permutation_importance(
    rfc, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X_train.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

Model accuracy score with 10 decision-trees : 0.9649


AttributeError: 'numpy.ndarray' object has no attribute 'columns'